In [24]:
#Import Libraries
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [25]:
"""
-------------------------- Constants --------------------------
arg_date - Designed Date that we will be using
src_format - String way of python to create year, month, and day
src_bucket - This is a bucket that we will be connecting from the S3, this is a public data base bucket
columns - These are the coluns that our report will have
key - This is the key
------------------------------------------------------------
"""
arg_date = '2021-10-22'
src_format ='%Y-%m-%d'
src_bucket = 'deutsche-boerse-xetra-pds'
trg_bucket = ''
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume']
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'


In [26]:
"""
-------------------------- Variables --------------------------
arg_date_dt - This will be the day befoe the target date
---------------------------------------------------------------
"""
arg_date_dt = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)

In [27]:
"""
--------------------------- Connection ---------------------------------

Purpose: Create the connection with AWS

Procedure:
        - Create the bucket connection
        - Create the bucket object
        - Create a list of the specific object you want to work with
          based on the date object that you want to work with.
          
"""
s3 = boto3.resource('s3')
bucket = s3.Bucket(src_bucket)
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0],'%Y-%m-%d').date() >= arg_date_dt]

In [28]:
"""
--------------------------- objects ---------------------------------
Purpose: Analyze the content of our data

Explanation of the Parameters of The Data:
        - Bucket name
        - Key: The key of every bucket is a combination of the "date+'0'
        
"""
objects

[s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-10-21/2021-10-21_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-10-21/2021-10-21_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-10-21/2021-10-21_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-10-21/2021-10-21_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-10-21/2021-10-21_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-10-21/2021-10-21_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-10-21/2021-10-21_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-10-21/2021-10-21_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-10-21/2021-10-21_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pd

In [29]:
"""
---------------------------- csv_to_df(filename) -------------------------
Function: csv_to_df(filename)

Purpose: Transform the csv file form the S3 bucket to a pandas data frame

Procedure:
        - For loop within the objects
            - Decode the csv file, making it unicode
            - Use StringIO to have this file as stream of strings
            - Convert it into a pandas dataframe
Parameters:
        filename, string, 

Return:

"""
def csv_to_df():
    df_all = pd.DataFrame()
    for obj in objects:
        csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
        data = StringIO(csv_obj)
        df = pd.read_csv(data, delimiter=',')
        df_all = df_all.append(df, ignore_index=True)
    return df_all

In [30]:
df_all = csv_to_df()

In [31]:
"""
--------------------------- Modify Table ---------------------------------
Purpose: Make the table look better

Procedure:
    - Only have the columns we want
    - Drop all the entries that are missing
    - The following tables are going to be created; 
        - Opening price per ISIN
        - Closing price per ISIN
        - Percent Change of Previous Closing
        
"""
df_all = df_all.loc[:,columns]
df_all.dropna(inplace=True)

## Get Opening Price per ISIN and Day

In [34]:

df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
df_all[df_all['ISIN']=='AT0000A0E9W5']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,AT0000A0E9W5,2021-10-21,07:00,21.84,21.84,21.82,21.82,1903,21.84
660,AT0000A0E9W5,2021-10-21,07:04,21.94,21.98,21.94,21.98,2096,21.84
3119,AT0000A0E9W5,2021-10-21,07:06,22.00,22.06,21.96,21.96,1159,21.84
3449,AT0000A0E9W5,2021-10-21,07:07,21.98,22.00,21.98,22.00,7572,21.84
4741,AT0000A0E9W5,2021-10-21,07:13,22.00,22.00,21.96,21.96,700,21.84
...,...,...,...,...,...,...,...,...,...
187274,AT0000A0E9W5,2021-10-22,15:26,21.90,21.90,21.90,21.90,670,21.76
187571,AT0000A0E9W5,2021-10-22,15:27,21.88,21.88,21.88,21.88,1242,21.76
187816,AT0000A0E9W5,2021-10-22,15:28,21.88,21.88,21.84,21.84,6508,21.76
188078,AT0000A0E9W5,2021-10-22,15:29,21.86,21.88,21.82,21.88,4142,21.76


## Get closing price per ISIN and day

In [35]:

df_all['closing_price'] =df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')
df_all[df_all['ISIN']=='AT0000A0E9W5']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
0,AT0000A0E9W5,2021-10-21,07:00,21.84,21.84,21.82,21.82,1903,21.84,21.80
660,AT0000A0E9W5,2021-10-21,07:04,21.94,21.98,21.94,21.98,2096,21.84,21.80
3119,AT0000A0E9W5,2021-10-21,07:06,22.00,22.06,21.96,21.96,1159,21.84,21.80
3449,AT0000A0E9W5,2021-10-21,07:07,21.98,22.00,21.98,22.00,7572,21.84,21.80
4741,AT0000A0E9W5,2021-10-21,07:13,22.00,22.00,21.96,21.96,700,21.84,21.80
...,...,...,...,...,...,...,...,...,...,...
187274,AT0000A0E9W5,2021-10-22,15:26,21.90,21.90,21.90,21.90,670,21.76,21.82
187571,AT0000A0E9W5,2021-10-22,15:27,21.88,21.88,21.88,21.88,1242,21.76,21.82
187816,AT0000A0E9W5,2021-10-22,15:28,21.88,21.88,21.84,21.84,6508,21.76,21.82
188078,AT0000A0E9W5,2021-10-22,15:29,21.86,21.88,21.82,21.88,4142,21.76,21.82


## Aggregations

In [36]:

df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT00000FACC2,2021-10-21,9.0000,9.020,8.900,9.0200,1060
1,AT00000FACC2,2021-10-22,9.0000,9.040,8.910,9.0400,933
2,AT0000606306,2021-10-21,24.7000,24.500,24.500,24.7600,1302
3,AT0000606306,2021-10-22,24.8400,25.000,24.840,25.0000,1110
4,AT0000609607,2021-10-21,14.2200,13.840,13.660,14.2200,415
...,...,...,...,...,...,...,...
6164,XS2284324667,2021-10-21,35.0740,34.608,34.608,35.1060,3114
6165,XS2284324667,2021-10-22,34.9000,34.920,34.900,35.2440,1661
6166,XS2314659447,2021-10-22,8.7705,8.583,8.583,8.7705,951
6167,XS2314660700,2021-10-21,17.8200,17.212,17.212,17.8200,595


## Percent Change Prev Closing

In [38]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100
df_all.drop(columns=['prev_closing_price'], inplace=True)
df_all = df_all.round(decimals=2)
df_all = df_all[df_all.Date >= arg_date]

## Write to S3

In [40]:
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket('xetra-123')
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='xetra-123', key='xetra_daily_report_20211024_132346.parquet')

## Reading the uploaded file

In [42]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20211023_230203.parquet
xetra_daily_report_20211024_130812.parquet
xetra_daily_report_20211024_132346.parquet


In [43]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20211023_230203.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT00000FACC2,2021-10-21,9.00,9.02,8.90,9.02,1060,-0.11
1,AT00000FACC2,2021-10-22,9.00,9.04,8.91,9.04,933,0.22
2,AT0000606306,2021-10-21,24.70,24.50,24.50,24.76,1302,-0.73
3,AT0000606306,2021-10-22,24.84,25.00,24.84,25.00,1110,2.04
4,AT0000609607,2021-10-21,14.22,13.84,13.66,14.22,415,-4.95
...,...,...,...,...,...,...,...,...
6164,XS2284324667,2021-10-21,35.07,34.61,34.61,35.11,3114,-1.42
6165,XS2284324667,2021-10-22,34.90,34.92,34.90,35.24,1661,0.90
6166,XS2314659447,2021-10-22,8.77,8.58,8.58,8.77,951,-5.10
6167,XS2314660700,2021-10-21,17.82,17.21,17.21,17.82,595,-2.48
